<a href="https://colab.research.google.com/github/nntadotzip/XLNet-sandbox/blob/main/XLNetQA_for_32formattedDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install and Import

In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install pytorch-transformers
!pip install datasets transformers

     |████████████████████████████████| 176 kB 4.3 MB/s 
     |████████████████████████████████| 131 kB 27.1 MB/s 
     |████████████████████████████████| 1.2 MB 32.6 MB/s 
     |████████████████████████████████| 895 kB 31.7 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 7.9 MB 32.8 MB/s 
     |████████████████████████████████| 138 kB 52.0 MB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 270 kB 4.3 MB/s 
     |████████████████████████████████| 2.8 MB 39.4 MB/s 
     |████████████████████████████████| 1.3 MB 40.3 MB/s 
     |████████████████████████████████| 243 kB 54.3 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 119 kB 52.1 MB/s 
     |████████████████████████████████| 636 kB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
     |████████████████████████████████| 142 kB 50.7 MB/s 
     |████████████████████████████████| 294 kB 52.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import transformers
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from transformers import XLNetModel, XLNetTokenizer, XLNetForQuestionAnswering
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [6]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: nntadotzip
Password: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [7]:
!git config --global credential.helper store

In [8]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [9]:
!apt install git-lfs
!git config --global user.email "nntanh@hcmiu.edu.vn"
!git config --global user.name "nntadotzip"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 2s (968 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


#Load Dataset

In [10]:
model_checkpoint = "xlnet-base-cased"
batch_size = 8

In [11]:
from datasets import load_dataset, load_metric
import json

In [12]:
# from google.colab import files
# uploaded = files.upload()

Saving QA_fromIUYouthVoice_test.csv to QA_fromIUYouthVoice_test.csv
Saving QA_fromIUYouthVoice_train.csv to QA_fromIUYouthVoice_train.csv


In [13]:
df_train = pd.read_csv('QA_fromIUYouthVoice_train.csv', encoding='utf8')
df_train
df_test = pd.read_csv('QA_fromIUYouthVoice_test.csv', encoding='utf8')
df_test

,answer,context,id,question,unit
0,"{'answer_start': [21], 'text': ['từ 07:00 sáng...","Căn tin IU hoạt động từ 07:00 sáng đến 14:00, ...",27,Canteen trường mình hoạt động từ mấy giờ đến m...,canteen
1,"{'answer_start': [13], 'text': ['được em nhé. ...",Tất nhiên là được em nhé. Căntin trường phục v...,28,"Em là k19, em có thắc mắc là liệu canteen trườ...",canteen
2,"{'answer_start': [46], 'text': ['vẫn hoạt động...","Chào bạn,\n Xe đưa đón sinh viên từ IU sang N...",29,Trong hiện tại và tương lai thì xe đưa đón sin...,hcth
3,"{'answer_start': [69], 'text': ['gửi email cho...","Chào em, em đến TTDVCNTT (P. A1.312) để kiểm t...",30,Dạ cho em hỏi là hồi đầu năm trường có phát Ma...,ttdvcntt
4,"{'answer_start': [39], 'text': ['chỉ dùng tron...",Tài khoản Office 365 cấp cho sinh viên chỉ dùn...,31,Tài khoản student email cũng như các ứng dụng ...,ttdvcntt
5,"{'answer_start': [286], 'text': ['Để lấy lại t...",Sinh viên được cung cấp tài khoản Office 365 t...,32,Cho em hỏi trường mình có cấp tài khoản micros...,ttdvcntt


In [14]:
df_train['answer'][0]

"{'answer_start': [16], 'text': ['P.KHTC O2.701 trong giờ hành chính']}"

In [15]:
df_train['answer'] = df_train['answer'].map(lambda x: json.loads(x.replace("'", "\"")))
df_test['answer'] = df_test['answer'].map(lambda x: json.loads(x.replace("'", "\"")))

In [16]:
df_train.to_json('train_json.json', orient='table', index=False)
df_test.to_json('test_json.json', orient='table', index=False)

In [17]:
datasets = load_dataset("json", data_files={'train': 'train_json.json', 'validation': 'test_json.json'}, field='data')

Using custom data configuration default-ac746b5cc7da016a


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ac746b5cc7da016a/0.0.0/d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
datasets

DatasetDict({
    train: Dataset({
        features: ['answer', 'context', 'id', 'question', 'unit'],
        num_rows: 26
    })
    validation: Dataset({
        features: ['answer', 'context', 'id', 'question', 'unit'],
        num_rows: 6
    })
})

In [19]:
print(datasets['train'])
print(datasets['validation'])

Dataset({
    features: ['answer', 'context', 'id', 'question', 'unit'],
    num_rows: 26
})
Dataset({
    features: ['answer', 'context', 'id', 'question', 'unit'],
    num_rows: 6
})


In [20]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [21]:
show_random_elements(datasets["train"])

,answer,context,id,question,unit
0,"{'answer_start': [34], 'text': ['em lấy đơn tại: https://onlinerequestoaa.hcmiu.edu.vn/ (Username: <mã sinh viên>, Password: <ddmmyyyy>), khi đã đầy đủ chữ ký, gửi mail về pdtdh@hcmiu.edu.vn']}","Để làm thủ tục thôi học/tạm dừng, em lấy đơn tại: https://onlinerequestoaa.hcmiu.edu.vn/ (Username: <mã sinh viên>, Password: <ddmmyyyy>), khi đã đầy đủ chữ ký, gửi mail về pdtdh@hcmiu.edu.vn",8,Anh chị cho em hỏi Em cũng giống bạn em đang bảo lưu sắp quá hạn nhưng vì lí do khách quan em không tiếp tục theo học trường mình được. Hiện giờ không lên trg làm thủ tục được thì mình còn cách nào khác không ạ ? Em cảm ơn,đtđh
1,"{'answer_start': [278], 'text': ['kéo dài thời gian đào tạo là tối đa 02 học kỳ']}","Thời gian tối đa hoàn thành chương trình đào tạo theo thiết kế là 1,5 lần thời gian thiết kế của chương trình đào tạo. Sinh viên đạt điều kiện xét tốt nghiệp về hoàn tất CTĐT và tích lũy đủ số tín chỉ quy định của CTĐT nhưng chưa đạt điều kiện chuẩn trình độ ngoại ngữ thì được kéo dài thời gian đào tạo là tối đa 02 học kỳ để bổ sung. Sinh viên không được đăng kí môn học trong thời gian này",12,Hôm nay vào edu thì em có thấy cái xử lý học vụ thì trong đó có cái mục là sv không hoàn thành CTDT trong thời gian cho phép. \n Nên em thắc mắc là sv được trễ thời hạn ra trường so với thời gian trong thẻ sinh viên là bao lâu ạ??,đtđh
2,"{'answer_start': [278], 'text': ['kéo dài thời gian đào tạo là tối đa 02 học kỳ']}","Thời gian tối đa hoàn thành chương trình đào tạo theo thiết kế là 1,5 lần thời gian thiết kế của chương trình đào tạo. Sinh viên đạt điều kiện xét tốt nghiệp về hoàn tất CTĐT và tích lũy đủ số tín chỉ quy định của CTĐT nhưng chưa đạt điều kiện chuẩn trình độ ngoại ngữ thì được kéo dài thời gian đào tạo là tối đa 02 học kỳ để bổ sung. Sinh viên không được đăng kí môn học trong thời gian này. Các vấn đề khác liên quan đến tốt nghiệp em mail về ttnquynh@hcmiu.edu.vn để được hỗ trợ",16,Theo em biết thì nếu học vượt quá học kì so với quy định khóa học thì sẽ bị hạ một bậc tốt nghiệp. Cho em hỏi thời gian quy định khóa học có phải là 6 năm ? Tức là nếu mình kéo dài không quá 4 học kì thì sẽ không bị hạ bậc tốt nghiệp đúng không ạ ?,đtđh
3,"{'answer_start': [12], 'text': ['email cho Phòng thông tin tài khoản của em (Số tài khoản, tên chủ tài khoản, ngân hàng, chi nhánh) về khtc@hcmiu.edu.vn']}","Chào em, em email cho Phòng thông tin tài khoản của em (Số tài khoản, tên chủ tài khoản, ngân hàng, chi nhánh) về khtc@hcmiu.edu.vn để Phòng cập nhật cho em nhé.",6,"Vào sem trước em có lên phòng Tài chính để cung cấp lại thông tin Tài khoản Ngân hàng của em, nhưng trường không cập nhật, nên sau đó đã gửi nhầm tiền vào tài khoản cũ em cung cấp trước đó. Hiện mùa dịch, em vẫn muốn cung cấp lại tài khoản mới của em cho nhà trường thì em phải làm thế nào ạ? Dạ em cảm ơn.",khtc
4,"{'answer_start': [12], 'text': ['đọc 2 thông báo liên quan tại đây: https://edusoftweb.hcmiu.edu.vn/default.aspx?page=chitietthongtin&id=1174']}",Em vui lòng đọc 2 thông báo liên quan tại đây:\n https://edusoftweb.hcmiu.edu.vn/default.aspx?page=chitietthongtin&id=1174\n \n https://edusoftweb.hcmiu.edu.vn/default.aspx?page=chitietthongtin&id=1179,11,"Em muốn hỏi các lớp IE có phải là các lớp tiên quyết không ạ? Tại vì em không tìm được nó trong danh sách. Em là sinh viên khóa K20, đã gap sem hè. Em muốn hỏi là sang năm 2 em có được phép đăng kí tín chỉ chuyên ngành và AE1 luôn không ạ.",đtđh
5,"{'answer_start': [108], 'text': ['liên hệ trực tiếp giảng viên hoặc nhờ hỗ trợ từ Khoa Bộ môn. Ngoài ra, trong thời gian đăng ký môn học, hệ thống sẽ không cập nhật điểm']}","Hiện nay điểm hoàn toàn được giảng viên nhập trực tiếp lên edusoftweb, vì vậy nếu em muốn biết điểm em phải liên hệ trực tiếp giảng viên hoặc nhờ hỗ trợ từ Khoa Bộ môn. Ngoài ra, trong thời gian đăng ký môn học, hệ thống sẽ không cập nhật điểm để bảo đảm tính ổn định khi đăng ký môn của sinh viên.",13,"Em là sinh viên khoa BA, cho em hỏi là khi nào trường sẽ hoàn tất việc n

#Preprocessing

In [22]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer =  transformers.XLNetTokenizerFast.from_pretrained(model_checkpoint)

# import transformers
assert isinstance(tokenizer, transformers.XLNetTokenizerFast)

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [23]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [24]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [25]:
len(tokenizer(example["question"], example["context"])["input_ids"])
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [26]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [27]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 300]

In [28]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

Theo em biet thi neu hoc vuot qua hoc ki so voi quy <unk>inh khoa hoc thi se bi ha mot bac tot nghiep. Cho em hoi thoi gian quy <unk>inh khoa hoc co phai la 6 nam? Tuc la neu minh keo dai khong qua 4 hoc ki thi se khong bi ha bac tot nghiep <unk>ung khong a?<sep> Thoi gian toi <unk>a hoan thanh chuong trinh <unk>ao tao theo thiet ke la 1,5 lan thoi gian thiet ke cua chuong trinh <unk>ao tao. Sinh vien <unk>at <unk>ieu kien xet tot nghiep ve hoan tat CT<unk>T va tich luy <unk>u so tin chi quy <unk>inh cua CT<unk>T nhung chua <unk>at <unk>ieu kien chuan trinh <unk>o ngoai ngu thi <unk>uoc keo dai thoi gian <unk>ao tao la toi <unk>a 02 hoc ky <unk>e bo sung. Sinh vien khong <unk>uoc <unk>ang ki mon hoc trong thoi gian nay. Cac van <unk>e khac lien quan <unk>en tot<sep><cls>
Theo em biet thi neu hoc vuot qua hoc ki so voi quy <unk>inh khoa hoc thi se bi ha mot bac tot nghiep. Cho em hoi thoi gian quy <unk>inh khoa hoc co phai la 6 nam? Tuc la neu minh keo dai khong qua 4 hoc ki thi se khon

In [29]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 3), (3, 4), (5, 7), (8, 9), (8, 11), (11, 12), (13, 14), (13, 16), (17, 18), (17, 19), (19, 20), (21, 23), (23, 24), (25, 26), (25, 27), (27, 29), (30, 31), (30, 33), (34, 36), (36, 37), (38, 39), (38, 40), (41, 43), (44, 45), (44, 46), (46, 47), (48, 49), (48, 50), (50, 51), (52, 53), (52, 53), (53, 55), (55, 56), (57, 58), (57, 59), (59, 61), (62, 64), (64, 65), (66, 67), (66, 69), (70, 71), (70, 72), (73, 75), (76, 78), (79, 80), (79, 82), (83, 84), (83, 86), (87, 89), (89, 90), (91, 92), (91, 92), (92, 95), (95, 97), (97, 98), (99, 102), (103, 105), (106, 108), (108, 109), (110, 111), (110, 113), (113, 114), (115, 116), (115, 119), (120, 121), (120, 122), (122, 123), (124, 125), (124, 125), (125, 127), (127, 128), (129, 130), (129, 131), (131, 133), (134, 136), (136, 137), (138, 140), (141, 142), (142, 145), (146, 148), (149, 150), (151, 152), (151, 154), (155, 156), (155, 156), (157, 159), (159, 160), (161, 163), (164, 165), (164, 166), (166, 167), (168, 169), (168, 171), (17

In [30]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

o o


In [31]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [32]:
answers = example["answer"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

297 323


In [33]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

keo dai thoi gian <unk>ao tao la toi <unk>a 02 hoc ky
kéo dài thời gian đào tạo là tối đa 02 học kỳ


In [34]:
pad_on_right = tokenizer.padding_side == "right"

In [35]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [36]:
features = prepare_train_features(datasets['train'][:5])

In [37]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#Finetuning

In [38]:
model = XLNetForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForQuestionAnswering: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnswering were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['answer_class.dense_0.bias', 'start_logits.dense.weight', 'end_logits.dense_0.weight', 'end_logits.LayerNorm.weight', 'end_logits.dense_0.bias', 'answer_class.dense_0.weight', 'start_logits.dense.bias', 'end_logits.dense_1.bias

In [39]:
from transformers import Trainer, TrainingArguments

In [40]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-IUYouthVoice-32",
)

In [41]:
from transformers import default_data_collator

data_collator = default_data_collator

In [42]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/nntadotzip/xlnet-base-cased-finetuned-IUYouthVoice-32 into local empty directory.


In [43]:
trainer.train()

***** Running training *****
  Num examples = 28
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log


***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
***** Running Evaluation *****
  Num examples = 6
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12, training_loss=4.602716128031413, metrics={'train_runtime': 23.3021, 'train_samples_per_second': 3.605, 'train_steps_per_second': 0.515, 'total_flos': 18290508687360.0, 'train_loss': 4.602716128031413, 'epoch': 3.0})

In [44]:
trainer.save_model("IUYouth-32-trained")

Saving model checkpoint to IUYouth-32-trained
Configuration saved in IUYouth-32-trained/config.json
Model weights saved in IUYouth-32-trained/pytorch_model.bin
tokenizer config file saved in IUYouth-32-trained/tokenizer_config.json
Special tokens file saved in IUYouth-32-trained/special_tokens_map.json


In [48]:
model.push_to_hub('xlnet-base-cased-finetuned-IUYouthVoice-32')

Cloning https://huggingface.co/nntadotzip/xlnet-base-cased-finetuned-IUYouthVoice-32 into local empty directory.
Configuration saved in xlnet-base-cased-finetuned-IUYouthVoice-32/config.json
Model weights saved in xlnet-base-cased-finetuned-IUYouthVoice-32/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.42k/454M [00:00<?, ?B/s]

To https://huggingface.co/nntadotzip/xlnet-base-cased-finetuned-IUYouthVoice-32
   83def53..870c0ce  main -> main



'https://huggingface.co/nntadotzip/xlnet-base-cased-finetuned-IUYouthVoice-32/commit/870c0ceb44e867938c6735ca84bdd90a9735bca6'

In [53]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [55]:
model_save_name = 'xlnet-base-cased-finetuned-IUYouthVoice-32.pt'
path = F"/content/gdrive/MyDrive/Colab Notebooks/save_models/{model_save_name}" 
torch.save(model.state_dict(), path)

#Evaluation

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

In [ ]:
output.start_logits.shape, output.end_logits.shape

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

In [ ]:
datasets["validation"][0]["answers"]

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

In [ ]:
trainer.push_to_hub()